# Descarga Automatica DDJJ CAPIV con Selenium

Se importan las librerias necesarias. 

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import pandas as pd
from numpy import random
from selenium.common import exceptions  

import glob
from itertools import islice
import win32com.client as win32

Se definen los parametros para el driver de selenium como la pagina de donde se realizara la descraga, el repositorio donde se guardaran los archivos, el path al driver, etc.

In [ ]:
my_url = f'https://www.se.gob.ar/datosupstream/consulta_avanzada/listado.php'
option = Options()
option.headless = False
prefs = {'download.default_directory' : r'C:\XXXXXXXXXXXXXXXXXXXX'} #---> se indica el directorio en donde se descargaran los archivos
option.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(options=option, executable_path=r'C:\XXXXXXXXXXXXXXXXXXXXXXXXX\chromedriver.exe')#--> se indica el path al dirver de selenium
driver.get(my_url)
#driver.maximize_window()

Se seleccionan los 3 despegables presentes en la pagina para ver su contenido. Como resultado se obtiene una lista con los nombres de cada empresa, otra con los meses y otra con los años que luego se utilizaran para iterar al momento de comenzar la descarga

In [3]:

#Para hacer click en las empresas inhabilitadas
#driver.find_element_by_xpath('//*[@id="ck_empresas_desabilitadas"]').click()

from selenium.webdriver.support.ui import Select
drp1 = Select(driver.find_element_by_xpath('//*[@id="contenedor-celeste-int"]/div/form/div[2]/select'))
drp2 = Select(driver.find_element_by_xpath('//*[@id="contenedor-celeste-int"]/div/form/div[3]/select[1]'))
drp3 = Select(driver.find_element_by_xpath('//*[@id="contenedor-celeste-int"]/div/form/div[3]/select[2]'))

all_options1 = drp1.options
all_options2 = drp2.options
all_options3 = drp3.options

empresas =[]
for option in all_options1:
    empresa = option.text
    empresas.append(empresa)

print("") 
print("")    
print(empresas)

drp2 = Select(driver.find_element_by_xpath('//*[@id="contenedor-celeste-int"]/div/form/div[3]/select[1]'))
all_options = drp2.options

meses =[]
for option in all_options2:
    mes = option.text
    meses.append(mes)

print("") 
print("") 
print(meses)

drp3 = Select(driver.find_element_by_xpath('//*[@id="contenedor-celeste-int"]/div/form/div[3]/select[2]'))
all_options3 = drp3.options

anos =[]
for option in all_options3:
    ano = option.text
    anos.append(ano)

print("") 
print("") 
print(anos)



['-> Seleccione una Empresa <-', 'ALIANZA PETROLERA ARGENTINA S.A.', 'AMERICAS PETROGAS ARGENTINA S.A.', 'APACHE ENERGIA ARGENTINA S.R.L.', 'APACHE PETROLERA ARGENTINA S.A.', 'APCO ARGENTINA INC.', 'APCO OIL AND GAS INTERNATIONAL INC (SUCURSAL ARGENTINA)', 'ARGENTA ENERGIA S.A.', 'CAPETROL ARGENTINA S.A.', 'CAPETROL S.A.', 'CAPEX S.A.', 'CENTRAL INTERNATIONAL CORPORATION (SUCURSAL ARGENTINA)', 'CGC ENERGIA SAU', 'CHAÑARES ENERGIA S.A.', 'CHEVRON ARGENTINA S.R.L.', 'CLEAR S.R.L.', 'COLHUE HUAPI S.A.', 'COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.', 'COMPAÑÍAS ASOCIADAS PETROLERAS S.A.', 'COPESA CIA CONSTRUCTORA PETROLERA SA', 'CRI HOLDING, INC.', 'CROWN POINT ENERGIA S.A.', 'CROWN POINT OIL & GAS S.A.', 'DAPETROL S.A.', 'EDHIPSA', 'EHRENCAP S.A.', 'ENAP SIPETROL ARGENTINA S.A.', 'ENARSA ENERGIA ARGENTINA S.A.', 'ENERGIA COMPAÑIA PETROLERA S.A.', 'ENERGIAL S.A.', 'ENERGICON S.A.', 'ENERGY OPERATIONS ARGENTINA LLC', 'ENI ARGENTINA EXPLORACION Y EXPLOTACION S.A.', 'EPP PETROLEO S.A.', 'EPSUR S.

In [32]:
print(len(empresas))
print(len(anos))

133
18


Como en principio la infomracion corresponde a 133 empresas desde el año 2006 puede llegar a haber 28728 archivos (133x18x12).
No todas las empresas operaron todos los años pero para acotar la descraga tanto en cantidad de archivos como en tiempo vamos a fijar un mes y un año en particular. 
En caso de querer descragar toda la informacion disponible sería cuestion de agregar un ciclo *for* para la lista de años y otro para la lista de meses

In [4]:
errores=[]

drp2.select_by_visible_text('Diciembre')
drp3.select_by_visible_text('2021')


for empresa in empresas:
    try: 
        drp1 = Select(driver.find_element_by_xpath('//*[@id="contenedor-celeste-int"]/div/form/div[2]/select'))    
        drp1.select_by_visible_text(empresa)

        sleep(random.uniform(2,4))

        elem = WebDriverWait(driver, 20).until(
             EC.element_to_be_clickable((By.XPATH,'//*[@id="contenedor-celeste-int"]/div/form/div[4]/input')))
        elem.click()

        sleep(random.uniform(2,4))

        elem = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH,'//*[@id="contenedor-celeste-int"]/div[2]/div/table/tbody/tr[2]/td[8]/a/img')))
        elem.click()
    except:
        errores.append(empresa)


In [34]:
print(len(errores))

0


Los archivos se descargan en formato .xls por lo tanto para poder trabajar con la libreria openpyxl y que sea mas sencillo trabajar en la unificacion se convierten todos los archivos a .xlsx. 

In [12]:
path = r'C:XXXXXXXX' #---> este directorio deberia ser el mismo en donde se descargan los archivos
all_files = glob.glob(path + "/*.xls") #--> se crea una lista con la ruta de cada archivo con extension .xls en el directorio 
print(len(all_files))

55


In [13]:
for filename in all_files:    
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(filename)

    wb.SaveAs(filename+"x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
    wb.Close()                               #FileFormat = 56 is for .xls extension
    excel.Application.Quit()


all_files_converted = glob.glob(path + "/*.xlsx") #--->se crea una lista con la ruta de cada archivo con extension .xlsx en el directorio 

A continuacion se abre cada archivo .xlsx , se extrae la informacion relevante y se estructura de forma tal que se pueda crear una base de datos con los datos de cada reporte

In [16]:
file=all_files_converted[0]
file
from openpyxl import load_workbook
wb = load_workbook(file)
ws = wb.active
data = list(ws.values)
cols= list(data[15])
cols.insert(0,'Empresa')
cols.insert(0,'Periodo')

lista_de_DDJJ=[]
lista_empresas=[]

from openpyxl import load_workbook
for file in all_files_converted:
    wb = load_workbook(file)
    ws = wb.active 
    data = ws.values
    empresa = ws['A3']
    periodo = ws['A4']
    registros=ws['A7']
    periodo = periodo.value.replace('Período: ','')
    empresa = empresa.value.replace('Empresa: ','')
    registros = int(registros.value.split()[-1])
    data = list(data)
    data = data[16:(16+registros)]
    df= pd.DataFrame(data)
    df.insert(0,-1,value=empresa)
    df.insert(0,-2,value=periodo)
    lista_de_DDJJ.append(df)
    #print(empresa)
    lista_empresas.append(empresa)


Se crea la base de datos y se exporta en formato .csv

In [26]:
database = pd.concat(lista_de_DDJJ)
database.columns=[cols]
database.to_csv('Dic_2021_CAPIV.csv', index=False)
